In [1]:
from langchain_groq import ChatGroq
import os
#from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain import HuggingFaceHub
#from sentence_transformers import SentenceTransformer
#from huggingface_hub import HuggingFaceHub
from langchain.llms import CTransformers

In [3]:
GROG_API_KEY = os.environ.get("GROG_API_KEY")
PINECONE_KEY = os.environ.get("PINECONE_KEY")
HUGGINGFACEHUB_API_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

In [4]:
PINECONE_KEY

'pcsk_dH9R3_PNhMeN3NWVGZ3GoN3XWrASbMdMLctr4qtwFGN37QzYoUAnCygmb6d6FoeXq3huP'

In [5]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.5,
    api_key=GROG_API_KEY,
    max_tokens=4096,
    max_retries=2
)

In [6]:
# Extract the PDF
def document_loader(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [7]:
extracted_data = document_loader("../data/")

In [8]:
# Create or split text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,
                                   chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_data=extracted_data)

In [10]:
# Download Embedding Model
#local_model_path = "../model/"
def download_hugging_face_embeddings(modelname):
    embeddings = HuggingFaceBgeEmbeddings(model_name=modelname,)
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings("sentence-transformers/all-MiniLM-L6-v2")

d:\LLMProject\Medical_Chat_Bot\mbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
query_result = embeddings.embed_query("Hello World")
print(f"Length : {len(query_result)}")

Length : 384


In [ ]:
# import pinecone
# pinecone.init(api_key=PINECONE_KEY,region="us-east-1")
# index_name = "medical-chatbot"

In [15]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_KEY)
pc_index = pc.Index("medical-chatbot")

In [ ]:
#docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],embedding=embeddings,index_name=pc_index)
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embeddings.embed_query(t.page_content)
   pc_index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        }
    ],
    namespace="real"
)